# ---------------------------------------------------------------------------
# Widget for Co-integration check (Vanguard ETFs)
# ---------------------------------------------------------------------------

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pandas_datareader as web
import datetime as datetime
import matplotlib.pyplot as plt
%matplotlib inline  
import arch.unitroot as at
import statsmodels.api as sm
import ipywidgets as widgets
# All vanguard's issued ETFs trading currently
list_of_etfs = ['VTI','VOO','VEA','VWO','BND','VNQ','VTV','VUG','VIG','BSV','VCSH','VEU',
                'VO','VB','VYM','VGK','VCIT','VGT','BIV','VBR','VV','VXUS','VT','BNDX','VOE',
                'VHT','VBK','VFH','VXF','VNQI','VOT','VPL','VMBS','VSS','VDE','VTIP','VDC','VIS',
                'MGK','VPU','BLV','VCR','VCLT','VAW','VGSH','MGV','VOOG','VTEB','VONG','VOX','VGIT',
                'MGC','VONV','VTWO','VWOB','VOOV','IVOO','VIOO','VONE','IVOG','IVOV','EDV','VIGI','VYMI',
                'VGLT','VTHR','VIOG','VIOV','VTWG','VTWV']
for etf in list_of_etfs:
    try:
        exec('{} = pd.DataFrame()'.format(etf))
        exec('{} = web.get_data_google("{}")'.format(etf,etf)) 
    except:
        pass
        
list_of_dfs = [VTI,VOO,VEA,VWO,BND,VNQ,VTV,VUG,VIG,BSV,VCSH,VEU,VO,VB,VYM,VGK,VCIT,VGT,BIV,VBR,VV,VXUS,
               VT,BNDX,VOE,VHT,VBK,VFH,VXF,VNQI,VOT,VPL,VMBS,VSS,VDE,VTIP,VDC,VIS,MGK,VPU,BLV,VCR,VCLT,
               VAW,VGSH,MGV,VOOG,VTEB,VONG,VOX,VGIT,MGC,VONV,VTWO,VWOB,VOOV,IVOO,VIOO,VONE,IVOG,IVOV,EDV,
               VIGI,VYMI,VGLT,VTHR,VIOG,VIOV,VTWG,VTWV]
cointegrated_etfs_close_price = pd.DataFrame(columns=list_of_etfs)
i = 0
for df in list_of_dfs:
    try:
        cointegrated_etfs_close_price[cointegrated_etfs_close_price.columns[i]] = df['Close']
    except:
        pass
    i = i+1  
    
cointegrated_etfs = pd.DataFrame(columns=list_of_etfs)
j = 0
for db in list_of_dfs:
    try:
        cointegrated_etfs[cointegrated_etfs.columns[j]] = np.log(db['Close']/db['Close'].shift(1))
    except:
        pass 
    j = j+1  
def cointegration_check(selected_etf):
    selected_etf = selected_etf[0]
    print("Correlation Facts of ETF prices:- ")
    print("========================================================================================================")
    print("")  
    for etf in list_of_etfs:
        if (etf==selected_etf):
            pass
        else:
            try:
                print("The correlation between",selected_etf,"and",etf,"is",cointegrated_etfs_close_price.loc[:,[str(selected_etf),str(etf)]].corr()[selected_etf][etf]*100,"%")
            except:
                pass
    print("")
    print("========================================================================================================")
    print("")
    print("Correlation Facts of ETF returns:- ")
    print("========================================================================================================")
    print("")
    for etf in list_of_etfs:
        if (etf==selected_etf):
            pass
        else:
            try:
                print("The correlation between",selected_etf,"and",etf,"returns is",cointegrated_etfs.loc[:,[str(selected_etf),str(etf)]].corr()[selected_etf][etf]*100,"%")
            except:
                pass
    print("")
    print("========================================================================================================")
    print("Cointegration Fact of ETFs :-")
    print("========================================================================================================")
    print("")
    print("Confidence level of 95%")
    print("-----------------------------------------------------------------------------")
    for etf in list_of_etfs:
        ols_result = sm.OLS(cointegrated_etfs_close_price[str(selected_etf)],cointegrated_etfs_close_price[str(etf)].astype(float)).fit()
        coeff = ols_result.params[0]
        error = ols_result.bse
        error = error.values[0]
        cointegrated_etfs_close_price['spread'] = cointegrated_etfs_close_price[str(selected_etf)] - coeff*cointegrated_etfs_close_price[str(etf)] - error
        spreaddf = at.ADF(cointegrated_etfs_close_price['spread'])
        if (spreaddf.pvalue<=0.05 and selected_etf!=etf):
            print(selected_etf,"and",etf,"have a strong cointegration with confidence level of 95%")    
        else:
            pass 
    print("")
    print("-----------------------------------------------------------------------------")
    print("Confidence level of 90%")
    print("-----------------------------------------------------------------------------")
    for etf in list_of_etfs:
        ols_result = sm.OLS(cointegrated_etfs_close_price[str(selected_etf)],cointegrated_etfs_close_price[str(etf)].astype(float)).fit()
        coeff = ols_result.params[0]
        error = ols_result.bse
        error = error.values[0]
        cointegrated_etfs_close_price['spread'] = cointegrated_etfs_close_price[str(selected_etf)] - coeff*cointegrated_etfs_close_price[str(etf)] - error
        spreaddf = at.ADF(cointegrated_etfs_close_price['spread'])
        if (spreaddf.pvalue<=0.1 and selected_etf!=etf):
            print(selected_etf,"and",etf,"have a strong cointegration with confidence level of 90% for a time period of",round((cointegrated_etfs_close_price.index[len(cointegrated_etfs_close_price)-1]-cointegrated_etfs_close_price.index[0]).days/365,3),"years")    
        else:
            pass 
widgets.interact(cointegration_check, selected_etf = {'VTI':['VTI'],'VOO':['VOO'],'VEA':['VEA'],'VWO':['VWO'],'BND':['BND'],
                                                      'VNQ':['VNQ'],'VTV':['VTV'],'VUG':['VUG'],'VIG':['VIG'],'BSV':['BSV'],
                                                      'VCSH':['VCSH'],'VEU':['VEU'],'VO':['VO'],'VB':['VB'],'VYM':['VYM'],
                                                      'VGK':['VGK'],'VCIT':['VCIT'],'VGT':['VGT'],'BIV':['BIV'],'VBR':['VBR'],
                                                      'VV':['VV'],'VXUS':['VXUS'],'VT':['VT'],'BNDX':['BNDX'],'VOE':['VOE'],
                                                      'VHT':['VHT'],'VBK':['VBK'],'VFH':['VFH'],'VXF':['VXF'],'VNQI':['VNQI'],
                                                      'VOT':['VOT'],'VPL':['VPL'],'VMBS':['VMBS'],
                                                      'VSS':['VSS'],'VDE':['VDE'],'VTIP':['VTIP'],'VDC':['VDC'],'VIS':['VIS'],'MGK':['MGK'],
                                                      'VPU':['VPU'],'BLV':['BLV'],'VCR':['VCR'],'VCLT':['VCLT'],'VAW':['VAW'],
                                                      'VGSH':['VGSH'],'MGV':['MGV'],'VOOG':['VOOG'],'VTEB':['VTEB'],'VONG':['VONG'],
                                                      'VOX':['VOX'],'VGIT':['VGIT'],'MGC':['MGC'],'VONV':['VONV'],'VTWO':['VTWO'],
                                                      'VWOB':['VWOB'],'VOOV':['VOOV'],'IVOO':['IVOO'],'VIOO':['VIOO'],'VONE':['VONE'],
                                                      'IVOG':['IVOG'],'IVOV':['IVOV'],'EDV':['EDV'],'VIGI':['VIGI'],'VYMI':['VYMI'],
                                                      'VGLT':['VGLT'],'VTHR':['VTHR'],'VIOG':['VIOG'],'VIOV':['VIOV'],'VTWG':['VTWG'],
                                                      'VTWV':['VTWV']})    

A Jupyter Widget

<function __main__.cointegration_check>